In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
import csv

In [2]:

# Initialize MediaPipe Holistic and Drawing utilities
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh  

In [3]:
class_name="Sad"

In [ ]:
# Open the webcam
capture = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while capture.isOpened():
        ret, frame = capture.read()
        if not ret:
            print("Failed to capture frame.")
            break

        # Flip the frame horizontally
        flip_frame = cv2.flip(frame, 1)

        # Convert the frame to RGB for processing
        image = cv2.cvtColor(flip_frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False  # Optimize by marking the image as read-only

        # Process the frame with MediaPipe Holistic
        results = holistic.process(image)

        # Convert the frame back to BGR for OpenCV display
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw face landmarks if detected
        if results.face_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.face_landmarks,
                mp_face_mesh.FACEMESH_CONTOURS,  
                mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1),
            )

        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.right_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
            
            )

        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,  
               
            )
        

        # Draw pose landmarks if detected
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=3),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
            )

        try:

            pose=results.pose_landmarks.landmark
            pose_row=list(np.array([ [landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

            face=results.face_landmarks.landmark
            face_row=list(np.array([ [landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in face]).flatten())
            
            row=pose_row+face_row
            row.insert(0,class_name)


            with open('cord.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)



            
        except:
            
            pass

    
            

        # Display the annotated frame
        cv2.imshow("Live Feed", image)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
capture.release()
cv2.destroyAllWindows()
